In [16]:
import numpy as np
import cv2
import imutils

In [17]:
# load the image and convert it to grayscale
def get_barcode(image_path):
    image = cv2.imread("D:/DTU-D/4/Fagprojekt/Data/photos/00a53a17-50db-4a3d-be1a-31ffd9537030/image.jpg")
    # image = cv2.imread(r"D:\DTU-D\4\Fagprojekt\Data\photos\00dc11b6-bb75-4b9b-95c1-b4f247796b25\20201120_0832291.jpg")
    # image = cv2.imread(r"D:\DTU-D\4\Fagprojekt\Data\photos\00e98217-20c2-429a-aa1a-b8c183a84d24\5AB1D358-F313-4107-9272-AA7D08D52804.jpeg")
    # image = cv2.imread(r"D:\DTU-D\4\Fagprojekt\Data\photos\0a000d87-9173-4bc3-a433-b77223437aa5\IMG_0329.JPG")
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # compute the Scharr gradient magnitude representation of the images
    # in both the x and y direction
    gradX = cv2.Sobel(gray, ddepth = cv2.CV_32F, dx = 1, dy = 0, ksize = -1)
    gradY = cv2.Sobel(gray, ddepth = cv2.CV_32F, dx = 0, dy = 1, ksize = -1)
    # subtract the y-gradient from the x-gradient
    gradient = cv2.subtract(gradX, gradY)
    gradient = cv2.convertScaleAbs(gradient)

    # blur and threshold the image
    blurred = cv2.blur(gradient, (9, 9))
    (_, thresh) = cv2.threshold(blurred, 225, 255, cv2.THRESH_BINARY)

    # construct a closing kernel and apply it to the thresholded image
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (21, 7))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # perform a series of erosions and dilations
    closed = cv2.erode(closed, None, iterations = 4)
    closed = cv2.dilate(closed, None, iterations = 4)

    # find the contours in the thresholded image, then sort the contours
    # by their area, keeping only the largest one
    cnts = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL,
                            cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = sorted(cnts, key = cv2.contourArea, reverse = True)[0]

    # compute the rotated bounding box of the largest contour
    rect = cv2.minAreaRect(c)
    box = cv2.cv.BoxPoints(rect) if imutils.is_cv2() else cv2.boxPoints(rect)
    box = np.int0(box)

    # draw a bounding box arounded the detected barcode and display the
    # image
    cv2.drawContours(image, [box], -1, (0, 255, 0), 3)
    # cv2.imshow("Image", image)
    # cv2.waitKey(0)
    cv2.imwrite("Image.jpg", image)
get_barcode("")